# Project 2 IMDB Classification

## Init 

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load Data
dataset=pd.read_csv('data/train_dataset.csv')
dataset.drop(columns=['id'], inplace=True)

In [ ]:
# Preview Data
dataset.shape
dataset.head()

In [ ]:
# Seperate dataset into features and classes
y = dataset['imdb_score_binned']
dataset.drop(columns=['imdb_score_binned'], inplace=True)
X = dataset

## Class Distribution

In [ ]:
# We will find the mean imdb score
mean_imdb_score = y.mean()
print(mean_imdb_score)

In [ ]:
# Making a histogram of count of imdb score
plt.rcParams.update({'font.size': 22})
plt.figure(figsize=(12,8))
print(y.name)

ax = sns.countplot(x=y.name, data=y.to_frame())
plt.title('Count of Movie Ratings')
plt.xlabel('IMDB Rating Binned')
plt.ylabel('Count')
total = y.count()
ax.bar_label(ax.containers[0], fmt=lambda x: f'{(x/total)*100:0.1f}%')
ax.margins(y=0.1)
plt.show()

## Data Preprocessing

### Normalisation

In [ ]:
# We will standardise some of the features. We will use z-score standardisation as it is better for zero mean models.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
# Get all numeric attributes except those that don't make sense to be standardised
numeric = X.select_dtypes(include='number')
numeric.drop(columns=['title_year', 'average_degree_centrality'], inplace=True)
numeric.columns

In [ ]:
plt.rcParams.update({'font.size': 12})
# Check distributions before normalisation
""" for c in numeric.columns:
    # Looking at the distribution of the duration of the movies
    plt.figure(figsize=(10,6))
    sns.histplot(X[c])
    plt.title(f'Distribution of {c}')
    plt.xlabel(c)
    plt.ylabel('Number of Movies')
    plt.show() """

In [ ]:
# Normalise the fields
for c in numeric.columns:
    X[c] = scaler.fit_transform(X[[c]])

In [ ]:
# Check fields are normalised
""" for c in numeric.columns:
    # Looking at the distribution of the duration of the movies
    plt.figure(figsize=(10,6))
    sns.histplot(X[c])
    plt.title(f'Distribution of {c}')
    plt.xlabel(c)
    plt.ylabel('Number of Movies')
    plt.show() """

### Correlation

In [ ]:
# We will do a correlation table between all the columns.
correlation = X.select_dtypes(include='number').corr()
# We will plot the correlation table
plt.figure(figsize=(12,8))
sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.title('Correlation Table')
plt.show()

### Removing Attributes

#### Removing Highly Correlated Attributes

In [ ]:
# cast_total_facebook_likes are correlated heavily with actor_1_facebook_likes, actor_2_facebook_likes, and actor_3_facebook_likes, so we drop the actor facebook likes
X.drop(columns=['actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes'], inplace=True)

# num_voted_users and num_user_for_reviews are highly correlated, so we drop the number of user reviews
X.drop(columns=['num_user_for_reviews'], inplace=True)

#### Removing Text Attributes

In [ ]:
# We will drop the id, director name, actor names, movie title columns, and plot keywords.
X.drop(columns=['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'movie_title', 'plot_keywords'], inplace=True)

# Removing title embedding
X.drop(columns=['title_embedding'], inplace=True)

#### Removing Attributes without much variation
These attributes have a lot of unique values, with one value being predominant

In [ ]:
X.columns

In [ ]:
""" for c in X.columns:
    plt.figure(figsize=(30,8))
    sns.countplot(x=c, data=X)
    plt.title(f'Count of Movies {c}')
    plt.xlabel(c)
    plt.ylabel('Count')
    plt.show() """

In [ ]:
X.drop(columns=['country', 'language', 'content_rating'], inplace=True)
X.columns

### Get doc2vec data for plot.

This is for vectorizing plot, basically a better bagofwords. If we want to use the movie's plot as a feature instead of useing words we should use this.

In [ ]:
doc2vec_plot_data = np.load('data/features_doc2vec/train_doc2vec_features_plot_keywords.npy')

We are assuming that if we have a movie that has a higly related plot point to a film in training dataset then the movie will have a similar score.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

doc2vec_scaled = scaler.fit_transform(doc2vec_plot_data)
# Now we create a similarity matrix using cosine similarity
similarity_matrix = cosine_similarity(doc2vec_scaled)

In [ ]:
# We will now create a matrix that is a matrix of difference of rating between movies
imdb_score_matrix = np.array(y)
imdb_score_matrix = imdb_score_matrix.reshape(-1,1)
imdb_score_matrix = np.repeat(imdb_score_matrix, len(imdb_score_matrix), axis=1)
imdb_score_matrix = np.abs(imdb_score_matrix - imdb_score_matrix.T)


In [ ]:
# Now we will scatter plot cosine similarity and imdb score difference
plt.figure(figsize=(12,8))
sns.scatterplot(x=similarity_matrix.flatten(), y=imdb_score_matrix.flatten())
plt.title('Cosine Similarity vs IMDB Score Difference')
plt.xlabel('Cosine Similarity')
plt.ylabel('IMDB Score Difference')
plt.show()


### One Hot Encoding Genres

In [ ]:
# We have a genre column which has multiple genres separated by '|'. We will split these genres into separate columns.
# We will first find all the unique genres
unique_genres = set()
for genre in X['genres']:
    genre_list = genre.split('|')
    for g in genre_list:
        unique_genres.add(g)
unique_genres = list(unique_genres)
print(unique_genres)
# Now we split the genres into separate columns
for genre in unique_genres:
    X[genre] = X['genres'].str.contains(genre).astype(int)
X.drop(columns=['genres'], inplace=True)
X.head()


In [ ]:
# We will also store the genres in a separate variable because we want to do stepwise selection for feature selection
genres = X[unique_genres]
X.drop(columns=unique_genres, inplace=True)
X.head()

## Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import classification_report
from sklearn.model_selection import learning_curve
from sklearn.model_selection import cross_val_score

# Assuming X and y are defined elsewhere in your code:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_genres = pd.concat([X, genres], axis=1)
# Adding genres to training set
X_train_genres = pd.concat([X_train, genres.iloc[X_train.index]], axis=1)
# Adding genres to test set
X_test_genres = pd.concat([X_test, genres.iloc[X_test.index]], axis=1)

In [ ]:
class Model:
    def __init__(self, estimator, param_grid):
        self.estimator = estimator
        self.param_grid = param_grid

    def grid_search(self, cv=5, n_jobs=-1, verbose=False, refit=True):
        self.gs = GridSearchCV(self.estimator, param_grid=self.param_grid, cv=cv, n_jobs=n_jobs, verbose=verbose, refit=refit)
    
    def get_best_model(self, X_train, y_train):
        self.gs.fit(X_train, y_train)
        self.best_estimator = self.gs.best_estimator_
        self.best_params = self.gs.best_params_
        self.best_cv = self.gs.best_score_
        return self.best_estimator, self.best_params, self.best_cv 

    def predict(self, X_test):
        y_pred = self.best_estimator.predict(X_test)
        return y_pred
    
    def sfs(self):
        self.sffs = SFS(self.best_estimator, k_features='best', forward=True, floating=False, cv=None, verbose = 2)
    
    def get_best_features(self, X_train, y_train):
        self.sffs = self.sffs.fit(X_train, y_train)
        best_subset = self.sffs.subsets_[max(self.sffs.subsets_, key=lambda k: self.sffs.subsets_[k]['avg_score'])]
        return list(best_subset['feature_names'])

    def fit_with_features(self, X_train, y_train, features=None):
        if features is None:
            self.best_estimator.fit(X_train, y_train)
        else: 
            self.best_estimator.fit(X_train[features], y_train)

    def get_learning_curve(self, X_train, y_train):
        self.train_sizes, self.train_scores, self.test_scores = learning_curve(self.best_estimator, X_train, y_train, cv=5)

        train_mean = np.mean(self.train_scores, axis=1)
        train_std = np.std(self.train_scores, axis=1)
        test_mean = np.mean(self.test_scores, axis=1)
        test_std = np.std(self.test_scores, axis=1)

        # Plot learning curves
        plt.plot(self.train_sizes, train_mean, label="Training score")
        plt.plot(self.train_sizes, test_mean, label="Cross-validation score")
        plt.fill_between(self.train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
        plt.fill_between(self.train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
        plt.xlabel("Training Set Size")
        plt.ylabel("Accuracy Score")
        plt.title("Learning Curves")
        plt.legend()
        plt.show()


## Model 1: Support Vector Machine

In [ ]:
from sklearn.svm import SVC

### Tuning Hyperparameters

In [ ]:
# SVM Model
param_grid = {
    'C': [1, 10],  # Regularization parameter
    'gamma': [0.01, 0.001],  # Kernel coefficient for RBF, polynomial and sigmoid
    'kernel': ['rbf', 'sigmoid']  # Type of kernel
}

svm = Model(SVC(), param_grid)
svm.grid_search()
svm_best, svm_params, svm_cv = svm.get_best_model(X_train, y_train)

In [ ]:
# Printing parameters for the best estimator including kernel)
print(svm_params)
y_pred_svm = svm.predict(X_test)
print(f'SVM Cross Validation Score: {svm_cv}')
print(classification_report(y_test, y_pred_svm))

### Feature Selection

In [ ]:
# Doing feature selection with the best SVM model
svm.sfs()
svm_best_features = svm.get_best_features(X_train, y_train)
print(svm_best_features)
svm.fit_with_features(X_train, y_train, svm_best_features)

In [ ]:
# Now checking accuracy with the best features
y_pred_svm_ft = svm.predict(X_test[svm_best_features])
svm_cv_scores = cross_val_score(svm_best, X_genres, y, cv=5)
print(f'SVM Cross Validation Score with Best Features: {svm_cv_scores.mean()}')
print(classification_report(y_test, y_pred_svm_ft))

### Adding Genres

In [ ]:
svm.best_estimator.fit(X_train_genres, y_train)
y_pred_svm = svm.best_estimator.predict(X_test_genres)
svm_cv_scores = cross_val_score(svm.best_estimator, X_genres, y, cv=5)
print(f'SVM Cross Validation Score with Genres: {svm_cv_scores.mean()}')
print(classification_report(y_test, y_pred_svm))

### Feature Selection with Genres

In [ ]:
""" # Doing feature selection with the best SVM model
svm.sfs()
#svm_best_features = svm.get_best_features(X_train_genres, y_train)
print(svm_best_features+unique_genres)
svm.fit_with_features(X_train_genres, y_train, svm_best_features+unique_genres) """

In [ ]:
""" # Now checking accuracy with the best features
y_pred_svm_ft = svm.predict(X_test_genres[svm_best_features+unique_genres])
svm_accuracy_ft = accuracy_score(y_test, y_pred_svm_ft)
print(f'SVM Accuracy with selected features: {svm_accuracy_ft}') """

In [ ]:
# Plotting learning curve
svm.get_learning_curve(X_genres, y)

## Model 2: Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

### Tuning Hyperparameters

In [ ]:
# Logistic Regression Model
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization strength
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],  # Types of penalties
    'solver': ['saga'],  # Solvers
    'max_iter': [100,200,1000]  # Number of iterations
}
logr_gs = Model(LogisticRegression(), param_grid)
logr_gs.grid_search()
logr_gs_best, logr_gs_params, logr_gs_cv = logr_gs.get_best_model(X_train, y_train)

In [ ]:
# Printing parameters for the best estimator including kernel)
print(logr_gs_params)
y_pred_logr_gs = logr_gs.predict(X_test)
print(f'Logistic Regression Cross Validation Score: {logr_gs_cv}')
print(classification_report(y_test, y_pred_logr_gs))

### Feature Selection

In [ ]:
# Doing feature selection with the best logr model
logr_gs.sfs()
logr_gs_best_features = logr_gs.get_best_features(X_train, y_train)
print(logr_gs_best_features)
logr_gs.fit_with_features(X_train, y_train, logr_gs_best_features)

In [ ]:
# Now checking accuracy with the best features
y_pred_logr_gs_ft = logr_gs.predict(X_test[logr_gs_best_features])
logr_gs_cv_scores = cross_val_score(logr_gs_best, X_genres, y, cv=5)
print(f'Logistic Regression Cross Validation Score with Best Features: {logr_gs_cv_scores.mean()}')
print(classification_report(y_test, y_pred_logr_gs_ft))

In [ ]:
# Plotting learning curve for logistic regression
logr_gs.get_learning_curve(X[logr_gs_best_features], y)

## Model 3: Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

### Tuning Hyperparameters

In [ ]:
# Random Forest Model
param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [10, 20, 30], 'max_features': ['auto', 'sqrt', 'log2']}
rf = Model(RandomForestClassifier(), param_grid)
rf.grid_search()
rf_best, rf_params, rf_cv = rf.get_best_model(X_train, y_train)

In [ ]:
# Printing parameters for the best estimator including kernel)
print(rf_params)
y_pred_rf = rf.predict(X_test)
print(f'Random Forest Cross Validation Score: {rf_cv}')
print(classification_report(y_test, y_pred_rf))

In [ ]:
# Plotting learning curve for random forest
rf.get_learning_curve(X, y)

### Feature Selection

In [ ]:
# Doing feature selection with the best random forest model
rf.sfs()
rf_best_features = rf.get_best_features(X_train, y_train)
print(rf_best_features)
rf.fit_with_features(X_train, y_train, rf_best_features)

In [ ]:
# Now checking accuracy with the best features
y_pred_rf_ft = rf.predict(X_test[rf_best_features])
rf_cv_scores = cross_val_score(rf_best, X, y, cv=5)
print(f'Random Forest Cross Validation Score with Best Features: {rf_cv_scores.mean()}')
print(classification_report(y_test, y_pred_rf_ft))

In [ ]:
# Plotting learning curve for random forest with best features
rf.get_learning_curve(X[rf_best_features], y)

In [ ]:
# Adding genres to training set
rf.best_estimator.fit(X_train_genres, y_train)
y_pred_rf = rf.best_estimator.predict(X_test_genres)
rf_cv_scores = cross_val_score(rf_best, X_genres, y, cv=5)
print(f'Random Forest Cross Validation Score with Best Features: {rf_cv_scores.mean()}')
print(classification_report(y_test, y_pred_rf))

In [ ]:
# Plotting learning curves for random forest with genres
rf.get_learning_curve(X_genres, y)

We can see that this model is overfitted.

## Model 4: Ensemble Model by Stacking

In [ ]:
from sklearn.ensemble import StackingClassifier

In [ ]:
estimators = [('rf', rf_best), ('svm', svm_best), ('logr', logr_gs_best)]
stack = StackingClassifier(estimators=estimators, final_estimator=logr_gs_best)
stack.fit(X_train_genres, y_train)
y_pred_stack = stack.predict(X_test_genres)
stack_cv_scores = cross_val_score(stack, X_genres, y, cv=5)
print(f'Stacking Classifier Cross Validation Score: {stack_cv_scores.mean()}')

In [ ]:
# Plotting learning curve for stacking
train_sizes, train_scores, test_scores = learning_curve(stack, X_genres, y, cv=5)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot learning curves
plt.plot(train_sizes, train_mean, label="Training score")
plt.plot(train_sizes, test_mean, label="Cross-validation score")
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
plt.xlabel("Training Set Size")
plt.ylabel("Accuracy Score")
plt.title("Learning Curves")
plt.legend()
plt.show()